In [1]:
%matplotlib inline

import gym
import itertools
import numpy as np
import os
import random
import sys
import tensorflow as tf

if "../" not in sys.path:
  sys.path.append("../")

from lib import plotting
from collections import deque, namedtuple

In [2]:
env = gym.envs.make("Breakout-v0")

[2017-02-23 10:53:09,467] Making new env: Breakout-v0


In [3]:
env.action_space.n

6

In [4]:
# Atari Actions: 0 (noop), 1 (fire), 2 (left) and 3 (right) are valid actions
VALID_ACTIONS = [0, 1, 2, 3]

In [5]:
class StateProcessor():
    """
    raw 픽셀을 리사이즈해준다
    Processes a raw Atari iamges. Resizes it and converts it to grayscale.
    """
    def __init__(self):
        # Build the Tensorflow graph
        with tf.variable_scope("state_processor"):
            self.input_state = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
            self.output = tf.image.rgb_to_grayscale(self.input_state) # RGB -> Gray Scale (210,160)
            self.output = tf.image.crop_to_bounding_box(self.output, 34, 0, 160, 160) #  CROP
            self.output = tf.image.resize_images(
                self.output,[84,84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR) # (84,84)로 리사이즈
            self.output = tf.squeeze(self.output)

    def process(self, sess, state):
        """
        Args:
            sess: A Tensorflow session object
            state: A [210, 160, 3] Atari RGB State

        Returns:
            A processed [84, 84, 1] state representing grayscale values.
        """
        return sess.run(self.output, { self.input_state: state })

# Preprocessing Code 분석 

In [27]:
test1 = env.reset()
test1.shape

(210, 160, 3)

In [39]:
input1 =  tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
gray = tf.image.rgb_to_grayscale(input1)
crop = tf.image.crop_to_bounding_box(gray, 34, 0, 160, 160)
poutput = tf.image.resize_images(
                    crop,[84,84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

output = tf.squeeze(poutput)

#tf.reset_default_graph()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    g = sess.run(gray, feed_dict = {input1 : test1})
    print(g.shape)  # gray scale
    c = sess.run(crop,{input1:test1})
    print(c.shape) # crop
    p = sess.run(poutput,{input1:test1})
    print(p.shape)
    o = sess.run(output,{input1:test1})
    print(o.shape)
    observation = np.stack([o] * 4, axis=2)
    print(observation.shape)

(210, 160, 1)
(160, 160, 1)
(84, 84, 1)
(84, 84)
(84, 84, 4)


In [43]:
np.stack([[[1,2,3]],[[2,3,4]]]*4,axis=2) # 왜 4배로 뿔리는거지?

array([[[1, 2, 1, 2, 1, 2, 1, 2],
        [2, 3, 2, 3, 2, 3, 2, 3],
        [3, 4, 3, 4, 3, 4, 3, 4]]])

#### tf.squeeze(input, axis=None, name=None, squeeze_dims=None) 

Removes dimensions of size 1 from the shape of a tensor.

중간중간 (불필요한) 1차원들을 지워준다.

In [44]:
class Estimator():
    """Q-Value Estimator neural network.
    
    Q-value를 근사하는 network (Q-value는 그 state에서 action을 했을 때 예상되는 value)
    Q-Network(행동 e-greedy), Target Nework(타겟 TD(0)해보고 greedy하게 가져가는) 
    
    This network is used for both the Q-Network and the Target Network.
    """

    def __init__(self, scope="estimator", summaries_dir=None):
        self.scope = scope
        # Writes Tensorboard summaries to disk
        self.summary_writer = None
        with tf.variable_scope(scope):
            # Build the graph
            self._build_model()
            if summaries_dir:
                summary_dir = os.path.join(summaries_dir, "summaries_{}".format(scope))
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                self.summary_writer =  writer = tf.summary.FileWriter(summary_dir, sess.graph)

    def _build_model(self):
        """
        Builds the Tensorflow graph.
        """

        
        # Input을 위한 placeholder 정의 
        
        # Our input are 4 RGB frames of shape 160, 160 each
        self.X_pl = tf.placeholder(shape=[None, 84, 84, 4], dtype=tf.uint8, name="X")
        # The TD target value (그 state에서 greedy하게 action을 취했을 때 얻어지는 값)
        self.y_pl = tf.placeholder(shape=[None], dtype=tf.float32, name="y")
        # 어떤 액션이 선택되었는지에 대한 값
        self.actions_pl = tf.placeholder(shape=[None], dtype=tf.int32, name="actions")
        
        # 인풋을 픽셀에 맞게 normalize 해주는 부분인듯!
        X = tf.to_float(self.X_pl) / 255.0
        batch_size = tf.shape(self.X_pl)[0] # batch_size는 None?? 상관없다??
        
        
        # 3 계층의 convolutional layers
        # input, num_outputs, kernel_size, stride , activation  // padding은 same? , pooling은? 
        conv1 = tf.contrib.layers.conv2d(
            X, 32, 8, 4, activation_fn=tf.nn.relu) 
        conv2 = tf.contrib.layers.conv2d(
            conv1, 64, 4, 2, activation_fn=tf.nn.relu)
        conv3 = tf.contrib.layers.conv2d(
            conv2, 64, 3, 1, activation_fn=tf.nn.relu)

        # Fully connected layers (히든 레이어 하나인듯)
        flattened = tf.contrib.layers.flatten(conv3) # conv layer 부분을 fc에 넣기 위해 펼친다 // 벡터로
        fc1 = tf.contrib.layers.fully_connected(flattened, 512)
        self.predictions = tf.contrib.layers.fully_connected(fc1, len(VALID_ACTIONS)) # 액션 4개로~
        
        # 여긴 뭐지? 좀 더 봐야 할듯
        # Get the predictions for the chosen actions only
        gather_indices = tf.range(batch_size) * tf.shape(self.predictions)[1] + self.actions_pl # batch의 각 instance마다 action의 index?
        self.action_predictions = tf.gather(tf.reshape(self.predictions, [-1]), gather_indices)

        # Target Error (Loss 정의)
        self.losses = tf.squared_difference(self.y_pl, self.action_predictions)
        self.loss = tf.reduce_mean(self.losses) # MSE

        # Optimizer Parameters from original paper
        self.optimizer = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6)
        self.train_op = self.optimizer.minimize(self.loss, global_step=tf.contrib.framework.get_global_step())
        
        # Summaries for Tensorboard
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.loss)
            tf.summary.histogram("loss_hist", self.losses)
            tf.summary.histogram("q_values_hist", self.predictions)
            tf.summary.scalar("max_q_value", tf.reduce_max(self.predictions))
            
            self.summaries = tf.summary.merge_all()


    def predict(self, sess, s):
        """
        Predicts action values.

        Args:
          sess: Tensorflow session
          s: State input of shape [batch_size, 4, 160, 160, 3]

        Returns:
          Tensor of shape [batch_size, NUM_VALID_ACTIONS] containing the estimated 
          action values.
        """
        return sess.run(self.predictions, { self.X_pl: s }) # s가 들어왔을 때 선택되는 action(predictions)

    def update(self, sess, s, a, y):
        """
        Updates the estimator towards the given targets.

        Args:
          sess: Tensorflow session object
          s: State input of shape [batch_size, 4, 160, 160, 3]
          a: Chosen actions of shape [batch_size]
          y: Targets of shape [batch_size]

        Returns:
          The calculated loss on the batch.
        """
        feed_dict = { self.X_pl: s, self.y_pl: y, self.actions_pl: a }
        summaries, global_step, _, loss = sess.run(
            [self.summaries, tf.contrib.framework.get_global_step(), self.train_op, self.loss],
            feed_dict)
        if self.summary_writer:
            self.summary_writer.add_summary(summaries, global_step)
        return loss

#### tf.range(start, limit=None, delta=1, dtype=None, name='range')

'start' is 3
'limit' is 18
'delta' is 3<br>
tf.range(start, limit, delta) ==> [3, 6, 9, 12, 15]

'start' is 3
'limit' is 1
'delta' is -0.5<br>
tf.range(start, limit, delta) ==> [3, 2.5, 2, 1.5]

'limit' is 5 <br>
tf.range(limit) ==> [0, 1, 2, 3, 4]|

#### tf.gather(params, indices, validate_indices=None, name=None)

In [48]:
# For Testing....

tf.reset_default_graph()
global_step = tf.Variable(0, name="global_step", trainable=False)

e = Estimator(scope="test")
sp = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Example observation batch
    observation = env.reset()
    
    observation_p = sp.process(sess, observation)
    observation = np.stack([observation_p] * 4, axis=2)
    observations = np.array([observation] * 2)
    
    print(observation_p.shape)
    print(observation.shape)
  #  print(observations.shape)
    

    action = np.random.choice(np.arange(4))
    next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
    next_state = sp.process(sess, next_state)
    print(next_state.shape)
    print(np.expand_dims(next_state, 2).shape)
    next_state = np.append(observation[:,:,1:], np.expand_dims(next_state, 2), axis=2)
    
    print(next_state.shape)
    
    
    # Test Prediction
    print(e.predict(sess, observations))

    # Test training step
    y = np.array([10.0, 10.0])
    a = np.array([1, 3])
    print(e.update(sess, observations, a, y))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


(84, 84)
(84, 84, 4)
(84, 84)
(84, 84, 1)
(84, 84, 4)
[[ 0.02584658  0.          0.02007666  0.04457932]
 [ 0.02584658  0.          0.02007666  0.04457932]]
99.5552


In [49]:
def copy_model_parameters(sess, estimator1, estimator2):
    """
    Copies the model parameters of one estimator to another.

    Args:
      sess: Tensorflow session instance
      estimator1: Estimator to copy the paramters from
      estimator2: Estimator to copy the parameters to
    """
    e1_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator1.scope)]
    e1_params = sorted(e1_params, key=lambda v: v.name)
    e2_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator2.scope)]
    e2_params = sorted(e2_params, key=lambda v: v.name)

    update_ops = []
    for e1_v, e2_v in zip(e1_params, e2_params):
        op = e2_v.assign(e1_v)
        update_ops.append(op)

    sess.run(update_ops)

#### tf.trainable_variables() 

Returns all variables created with trainable=True.

When passed trainable=True, the Variable() constructor automatically adds new variables to the graph collection GraphKeys.TRAINABLE_VARIABLES. This convenience function returns the contents of that collection.

In [50]:
def make_epsilon_greedy_policy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.

    Args:
        estimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment.

    Returns:
        A function that takes the (sess, observation, epsilon) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(sess, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))[0]
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [51]:
def deep_q_learning(sess,
                    env,
                    q_estimator,
                    target_estimator,
                    state_processor,
                    num_episodes,
                    experiment_dir,
                    replay_memory_size=500000,
                    replay_memory_init_size=50000,
                    update_target_estimator_every=10000,
                    discount_factor=0.99,
                    epsilon_start=1.0,
                    epsilon_end=0.1,
                    epsilon_decay_steps=500000,
                    batch_size=32,
                    record_video_every=50):
    """
    Q-Learning algorithm for fff-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.

    Args:
        sess: Tensorflow Session object
        env: OpenAI environment
        q_estimator: Estimator object used for the q values
        target_estimator: Estimator object used for the targets
        state_processor: A StateProcessor object
        num_episodes: Number of episodes to run for
        experiment_dir: Directory to save Tensorflow summaries in
        replay_memory_size: Size of the replay memory
        replay_memory_init_size: Number of random experiences to sampel when initializing 
          the reply memory.
        update_target_estimator_every: Copy parameters from the Q estimator to the 
          target estimator every N steps
        discount_factor: Lambda time discount factor
        epsilon_start: Chance to sample a random action when taking an action.
          Epsilon is decayed over time and this is the start value
        epsilon_end: The final minimum value of epsilon after decaying is done
        epsilon_decay_steps: Number of steps to decay epsilon over
        batch_size: Size of batches to sample from the replay memory
        record_video_every: Record a video every N episodes

    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    # 상태 전이를 표현할 튜플
    Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

    # The replay memory
    replay_memory = []

    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))

    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "ckps")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    monitor_path = os.path.join(experiment_dir, "monitor")

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)

    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
    
    # Get the current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())

    # epsilon이 시간에 따라 decay되도록 term을 가지고 배열로 만들어 놓기 (시작값, 끝값, 간격)
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)

    # 행동 policy < e-greedy >
    policy = make_epsilon_greedy_policy(
        q_estimator,
        len(VALID_ACTIONS))

    # 초기 experience들을 replay memory가 꽉 찰 때까지 넣어준다.
    print("Populating replay memory...")
    state = env.reset() # 210*160*3
    state = state_processor.process(sess, state) # 84*84
    state = np.stack([state] * 4, axis=2) # 84*84*4
    for i in range(replay_memory_init_size):
        action_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs) # 액션 선택
        next_state, reward, done, _ = env.step(VALID_ACTIONS[action]) # 다음 스텝 해보기
        next_state = state_processor.process(sess, next_state)
        next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2) # 84*84*4
        replay_memory.append(Transition(state, action, reward, next_state, done))
        if done:
            state = env.reset()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)
        else:
            state = next_state

    # Record videos
   # gym.wrappers.Monitor(env, monitor_path)
#     env.monitor.start(monitor_path,
#                       resume=True,
#                       video_callable=lambda count: count % record_video_every == 0)

    for i_episode in range(num_episodes):

        # Save the current checkpoint
        saver.save(tf.get_default_session(), checkpoint_path)

        # Reset the environment
        state = env.reset()
        state = state_processor.process(sess, state)
        state = np.stack([state] * 4, axis=2)
        loss = None

        # One step in the environment
        for t in itertools.count():

            # Epsilon for this time step
            epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]

            # Add epsilon to Tensorboard
            episode_summary = tf.Summary()
            episode_summary.value.add(simple_value=epsilon, tag="epsilon")
            q_estimator.summary_writer.add_summary(episode_summary, total_t)

            # Maybe update the target estimator
            if total_t % update_target_estimator_every == 0:
                copy_model_parameters(sess, q_estimator, target_estimator) # 일정 step마다 target_network update
                print("\nCopied model parameters to target network.")
            
            # Print out which step we're on, useful for debugging.
            sys.stdout.write("\rStep {} ({}) @ Episode {}/{}, loss: {}".format(
                    t, total_t, i_episode + 1, num_episodes, loss))
            sys.stdout.flush()

            # Take a step
            action_probs = policy(sess, state, epsilon)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs) # 액션 선택
            next_state, reward, done, _ = env.step(VALID_ACTIONS[action]) #한번 가보고
            next_state = state_processor.process(sess, next_state)
            next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2) # 84*84*4

            # 꽉 차면 오래된 기억부터 팝
            if len(replay_memory) == replay_memory_size:
                replay_memory.pop(0)

            # 새 기억 집어넣고
            replay_memory.append(Transition(state, action, reward, next_state, done))   

            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t

            # replay_memory에서 배치사이즈만큼을 샘플링한다!
            samples = random.sample(replay_memory, batch_size)
            states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*samples)) 
            # 각 step 별로 [state,action,reward, next_state,done]으로 묶여있던거를 np.array로 타입 바꾸고 병렬적으로 쫙 풀어서 나눠담기

            # Calculate q values and targets
            q_values_next = target_estimator.predict(sess, next_states_batch)
            targets_batch = reward_batch + np.invert(done_batch).astype(np.float32) * discount_factor * np.amax(q_values_next, axis=1)
            # 타겟 value!! target_estimator로 예측한... greedy target!!!
            
            
            # Perform gradient descent update
            states_batch = np.array(states_batch) # 형변환 왜 또 하는거지..?
            # Loss를 reduce_mean을 통해 scalar로 받아오고, Optimizing까지 해준다. 
            loss = q_estimator.update(sess, states_batch, action_batch, targets_batch)

            if done:
                break

            state = next_state
            total_t += 1 # global step??

        # Add summaries to tensorboard
        episode_summary = tf.Summary()
        episode_summary.value.add(simple_value=stats.episode_rewards[i_episode], node_name="episode_reward", tag="episode_reward")
        episode_summary.value.add(simple_value=stats.episode_lengths[i_episode], node_name="episode_length", tag="episode_length")
        q_estimator.summary_writer.add_summary(episode_summary, total_t)
        q_estimator.summary_writer.flush()

        yield total_t, plotting.EpisodeStats(
            episode_lengths=stats.episode_lengths[:i_episode+1],
            episode_rewards=stats.episode_rewards[:i_episode+1])

#     env.monitor.close()
    return stats

In [52]:
tf.reset_default_graph()

# Where we save our checkpoints and graphs
experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
    
# Create estimators
q_estimator = Estimator(scope="q", summaries_dir=experiment_dir)
target_estimator = Estimator(scope="target_q")

# State processor
state_processor = StateProcessor()

# Run it!
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                    env,
                                    q_estimator=q_estimator,
                                    target_estimator=target_estimator,
                                    state_processor=state_processor,
                                    experiment_dir=experiment_dir,
                                    num_episodes=10000,
                                    replay_memory_size=500000,
                                    replay_memory_init_size=50000,
                                    update_target_estimator_every=10000,
                                    epsilon_start=1.0,
                                    epsilon_end=0.1,
                                    epsilon_decay_steps=500000,
                                    discount_factor=0.99,
                                    batch_size=32):

        print("\nEpisode Reward: {}".format(stats.episode_rewards[-1]))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Populating replay memory...

Copied model parameters to target network.
Step 175 (175) @ Episode 1/10000, loss: 0.00033073188387788833
Episode Reward: 0.0
Step 160 (335) @ Episode 2/10000, loss: 0.03188493102788925017
Episode Reward: 0.0
Step 210 (545) @ Episode 3/10000, loss: 0.00020879003568552434
Episode Reward: 1.0
Step 347 (892) @ Episode 4/10000, loss: 0.00025085199740715325
Episode Reward: 4.0
Step 264 (1156) @ Episode 5/10000, loss: 0.00017177494009956717
Episode Reward: 1.0
Step 431 (1587) @ Episode 6/10000, loss: 5.192850949242711e-056
Episode Reward: 4.0
Step 214 (1801) @ Episode 7/10000, loss: 0.00031539093470200896
Episode Reward: 1.0
Step 205 (2006) @ Episode 8/10000, loss: 8.935783989727497e-055
Episode Reward: 1.0
Step 316 (2322) @ Episode 9/10000, loss: 8.901790715754032e-057
Episode Reward: 3.0
Step 163 (2485) @ Episode 10/10000, loss: 0.00014565957826562226
Episode Reward: 0.0
Step 302 (2787) @ Episode 11/10000, loss: 7.081223884597421e-059
Episode Reward: 2.0
Step 1

KeyboardInterrupt: 